<a href="https://colab.research.google.com/github/Santapaji/NBC_for_rotten_tomatoes/blob/main/DM_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import re
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# load stop words
stop_words = set(stopwords.words('english'))

# initialize Porter stemmer
stemmer = PorterStemmer()

# initialize vocabulary
vocab = defaultdict(int)

# load data
data = []
with open('data/rt_reviews.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        label = row['Freshness']
        text = row['Review']
        
        # tokenize text
        words = re.findall(r'\b\w+\b', text)
        
        # remove stop words and stem words
        words = [stemmer.stem(word.lower()) for word in words if word.lower() not in stop_words]
        
        # update vocabulary
        for word in words:
            vocab[word] += 1
        
        # add labeled data
        data.append((words, label))

# build vocabulary
vocab = {word: i for i, (word, count) in enumerate(vocab.items()) if count >= 5}

# build reverse index
index = {i: word for word, i in vocab.items()}


In [ ]:
import math

# initialize prior probabilities
num_fresh = sum(1 for _, label in data if label == 'Fresh')
num_rotten = sum(1 for _, label in data if label == 'Rotten')
prior_fresh = num_fresh / len(data)
prior_rotten = num_rotten / len(data)

# initialize conditional probabilities
cond_fresh = defaultdict(float)
cond_rotten = defaultdict(float)

# calculate conditional probabilities
for words, label in data:
    for word in words:
        if word in vocab:
            if label == 'Fresh':
                cond_fresh[word] += 1
            else:
                cond_rotten[word] += 1

# normalize conditional probabilities
for word in vocab:
    cond_fresh[word] = (cond_fresh[word] + 1) / (num_fresh + 2)
    cond_rotten[word] = (cond_rotten[word] + 1) / (num_rotten + 2)

# define predict function
def predict(words):
    # calculate probability of Fresh
    prob_fresh = math.log(prior_fresh)
    for word in words:
        if word in vocab:
            prob_fresh += math.log(cond_fresh[word])
    
    # calculate probability of Rotten
    prob_rotten = math.log(prior_rotten)
    for word in words:
        if word in vocab:
            prob_rotten += math.log(cond_rotten[word])
    
    # return class with higher probability
    if prob_fresh > prob_rotten:
        return 'Fresh'
    else:
        return 'Rotten'


In [ ]:
# load dev data
dev_data = []
with open('data/rotten_tomatoes_dev.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        label = row['Freshness']
        text = row['Review']
        
        # tokenize


In [ ]:
# tokenize dev data
for text, label in dev_raw:
    words = tokenize(text)
    dev_data.append((words, label))

# evaluate performance on dev dataset
num_correct = 0
for words, label in dev_data:
    predicted_label = predict(words)
    if predicted_label == label:
        num_correct += 1

accuracy = num_correct / len(dev_data)
print('Accuracy on dev dataset:', accuracy)
